# Fashion MNIST Image Classification - Azure ML SDK Deployment

In this Fashion MNIST notebook we introduce how to take the models you have registered and versioned, deploy them to production and monitor them 

This code will show how Azure ML SDK can support your machine learning project with:
* creating a training and scoring script for your model
* creating a bespoke environment for your model to run in (libraries and packages to install)
* setting up a container service to host your model
* deploying your model to the cloud container service
* retrieving the service scoring URL for your hosted model
* sending test data to your hosted model to be scored and returned to compare for the accuracy of your model on your validation set

This notebook is based off the great documentation tutorial here: [https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-deploy-models-with-aml](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-deploy-models-with-aml?WT.mc_id=aisummit-github-amynic)

## Import packages 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Load workspace and download registered model 

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model=Model(ws, 'keras_dl_fashion_test')
model.download(target_dir = '.', exist_ok = 'True')
import os 
# verify the downloaded model file
os.stat('model.h5')

## Load and format Test dataset

In [ ]:
import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import backend as K
print(os.environ['KERAS_BACKEND'])

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
import matplotlib.pyplot as plt
from utils import load_data

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#no. of classes
num_classes = 10
# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

x_test = x_test.astype('float32')
x_test /= 255
y_test = utils.to_categorical(y_test,  num_classes)

#X_test = load_data('./data/test-images.gz', False) / 255.0
#y_test = load_data('./data/test-labels.gz', True).reshape(-1)

## Create a score.py script

In [ ]:
%%writefile score.py
import keras as K
from azureml.core.model import Model
from keras.models import Sequential
from keras.models import load_model
import h5py
import sys
import os
import numpy as np
import json

def init():
    global model
    print("Python version: " + str(sys.version) + ", keras version: " + K.__version__)
    print("Executing init() method...")
    model_path = Model.get_model_path('model.h5')
    print("got model...")
    model = K.models.load_model(model_path)
    print("loaded model...")

def run(raw_data):
    print("Executing run(raw_data) method...")
    data = np.array(json.loads(raw_data)['data'])
    data = np.reshape(data, (30,28,28,1))
    y_hat = model.predict(data)
    print("Eexcuted predictions...")
    return json.dumps(y_hat.tolist())

## Create a bespoke environment to run your model in 

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("keras")
myenv.add_tensorflow_pip_package(core_type='cpu', version='1.11.0')

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

## Setup a Container service to host your model in

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "fashion-MNIST",  "method" : "keras"}, 
                                               description='Deep Learning to classify fashion clothing items, using Keras')

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Deep Learning to classify fashion clothing items, using Keras",
                                                  tags = {"data": "fashionmnist", "type": "classification"}
                                                 )

#the service name must not already exist in your workspace, else rename it 
service_name = 'deployfashion'
service = Webservice.deploy(deployment_config = aciconfig,
                                image_config = image_config,
                                model_paths = ['model.h5'],
                                name = service_name,
                                workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
print(service.scoring_uri)

## Test your model by calling the hosted API with test data

In [ ]:
import json
s = 100
e = s + 30

#sample_indices = (x_test[0:n])

test_samples = json.dumps({"data": x_test[s:e].tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

test_labels = y_test[s:e]

##testing input correct
body = np.array(json.loads(test_samples)['data'])

# predict using the deployed model
result = json.loads(service.run(input_data=test_samples))

for i in range(0, len(result)):
    indice_prediction = np.argmax(result[i])
    indice_label = np.argmax(test_labels[i])
    if indice_prediction == indice_label:
        print("correct prediction for sample " + str(i))
    else:
        print("INCORRECT prediction for sample" + str(i))